# Solving triangular systems

It's easy to get just the lower triangular part of any matrix using the `tril` command.

In [1]:
A = rand(1.:9.,5,5)
using LinearAlgebra
L = tril(A)

5×5 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0
 5.0  4.0  0.0  0.0  0.0
 8.0  8.0  3.0  0.0  0.0
 7.0  2.0  4.0  7.0  0.0
 8.0  1.0  6.0  5.0  7.0

We'll set up and solve a linear system with this matrix.

In [2]:
b = ones(5)
using FundamentalsNumericalComputation
x = FNC.forwardsub(L,b)

5-element Array{Float64,1}:
  1.0
 -1.0
  0.3333333333333333
 -0.7619047619047619
 -0.5986394557823129

It's not clear what the error in this answer is. However, the residual, while not zero, is comparable to $\varepsilon_\text{mach}$ in size.

In [3]:
b - L*x

5-element Array{Float64,1}:
  0.0
  0.0
  0.0
 -8.881784197001252e-16
  0.0

Next we'll engineer a problem to which we know the exact answer.

In [4]:
alpha = 0.3;
beta = 2.2;
U = diagm(0=>ones(5),1=>[-1,-1,-1,-1])
U[1,[4,5]] = [ alpha-beta, beta ]
U

5×5 Array{Float64,2}:
 1.0  -1.0   0.0  -1.9   2.2
 0.0   1.0  -1.0   0.0   0.0
 0.0   0.0   1.0  -1.0   0.0
 0.0   0.0   0.0   1.0  -1.0
 0.0   0.0   0.0   0.0   1.0

In [5]:
x_exact = ones(5)
b = [alpha,0,0,0,1]

x = FNC.backsub(U,b)
err = x - x_exact

5-element Array{Float64,1}:
 2.220446049250313e-16
 0.0
 0.0
 0.0
 0.0

Everything seems OK here. But another example, with a different value for $\beta$, is more troubling.

In [6]:
alpha = 0.3;
beta = 1e12;
U = diagm(0=>ones(5),1=>[-1,-1,-1,-1])
U[1,[4,5]] = [ alpha-beta, beta ]
b = [alpha,0,0,0,1]

x = FNC.backsub(U,b)
err = x - x_exact

5-element Array{Float64,1}:
 -4.882812499995559e-5
  0.0
  0.0
  0.0
  0.0

It's not so good to get four digits of accuracy after starting with sixteen! But the source of the error is not hard to track down. Solving for $x_1$ performs $(\alpha-\beta)+\beta$ in the first row. Since $|\alpha|$ is so much smaller than $|\beta|$, this a recipe for losing digits to subtractive cancellation.